<a href="https://colab.research.google.com/github/santhanalakshmi04/DEEP_ACTIVITY/blob/main/DEEP_ACT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
NAME : SANTHANA LAKSHMI K
REG NO : 212222240091

Provide a brief explanation of the architecture and working process of a GAN, followed by a simple Python code example using PyTorch on the MNIST dataset



Architecture and Working Process of a GAN
A Generative Adversarial Network (GAN) is a framework composed of two neural networks, trained simultaneously through adversarial processes:

Generator (G):

Takes random noise as input and generates data resembling the real dataset.

Its goal is to fool the Discriminator into classifying its output as real.

Discriminator (D):

Takes data samples and tries to distinguish between real data (from the dataset) and fake data (from the Generator).

Its goal is to correctly classify real vs. fake inputs.

Training Process:

The Generator and Discriminator are trained together in a loop.

Discriminator is trained to maximize the probability of assigning the correct label to both real and fake samples.

Generator is trained to minimize the probability that the Discriminator correctly identifies its outputs as fake.

This leads to a minimax game where the Generator improves to produce more realistic data, and the Discriminator becomes better at detection.

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:
# Hyperparameters
latent_size = 64
hidden_size = 256
image_size = 28 * 28
batch_size = 100
num_epochs = 50
learning_rate = 0.0002

In [4]:
# MNIST data loader
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [5]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
)

In [6]:
# Generator
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh()
)

In [7]:
# Loss and optimizers
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate)
g_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate)

In [8]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
D, G = D.to(device), G.to(device)

In [9]:
# Training loop
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(dataloader):
        # Flatten images
        images = images.view(batch_size, -1).to(device)

        # Labels
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Train Discriminator
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)

        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images.detach())
        d_loss_fake = criterion(outputs, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        # Train Generator
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        g_loss = criterion(outputs, real_labels)

        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}')


Epoch [1/50], d_loss: 1.5046, g_loss: 0.6138
Epoch [2/50], d_loss: 0.9335, g_loss: 1.1116
Epoch [3/50], d_loss: 0.7692, g_loss: 1.2611
Epoch [4/50], d_loss: 0.9564, g_loss: 1.2394
Epoch [5/50], d_loss: 1.0760, g_loss: 1.1503
Epoch [6/50], d_loss: 0.8248, g_loss: 1.2817
Epoch [7/50], d_loss: 1.1150, g_loss: 1.0391
Epoch [8/50], d_loss: 0.6740, g_loss: 1.5675
Epoch [9/50], d_loss: 1.0489, g_loss: 1.2985
Epoch [10/50], d_loss: 0.5335, g_loss: 1.6943
Epoch [11/50], d_loss: 0.9098, g_loss: 1.2413
Epoch [12/50], d_loss: 0.9682, g_loss: 1.1007
Epoch [13/50], d_loss: 1.0468, g_loss: 1.7855
Epoch [14/50], d_loss: 1.3743, g_loss: 0.9475
Epoch [15/50], d_loss: 1.1900, g_loss: 0.9802
Epoch [16/50], d_loss: 0.9104, g_loss: 1.6892
Epoch [17/50], d_loss: 0.6465, g_loss: 2.0010
Epoch [18/50], d_loss: 1.5691, g_loss: 0.9755
Epoch [19/50], d_loss: 1.0015, g_loss: 1.1269
Epoch [20/50], d_loss: 1.2013, g_loss: 1.2140
Epoch [21/50], d_loss: 0.7100, g_loss: 1.6279
Epoch [22/50], d_loss: 0.9273, g_loss: 1.24